In [ ]:
import numpy as np
from HH4b import utils
from HH4b import postprocessing
import xgboost as xgb
import importlib
import hist
import os
import mplhep as hep
import matplotlib.pyplot as plt
from HH4b.postprocessing.PostProcess import add_bdt_scores
import HH4b

plt.style.use(hep.style.ROOT)
from HH4b.hh_vars import txbbsfs_decorr_pt_bins, txbbsfs_decorr_txbb_wps
import json

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
package_path = os.path.dirname(HH4b.__file__)
legacy_label = "Legacy"
mass_var = "H2PNetMass"
bdt_model_name = "24May31_lr_0p02_md_8_AK4Away"
bdt_config = "24May31_lr_0p02_md_8_AK4Away"


def get_dataframe(events_dict, year, bdt_model_name, bdt_config):
    bdt_model = xgb.XGBClassifier()
    bdt_model.load_model(
        fname=f"{package_path}/boosted/bdt_trainings_run3/{bdt_model_name}/trained_bdt.model"
    )
    make_bdt_dataframe = importlib.import_module(
        f".{bdt_config}", package="HH4b.boosted.bdt_trainings_run3"
    )

    bdt_events_dict = {}
    for key in events_dict:
        events = events_dict[key]
        bdt_events = make_bdt_dataframe.bdt_dataframe(events)
        preds = bdt_model.predict_proba(bdt_events)
        # inference
        add_bdt_scores(bdt_events, preds)

        # extra variables
        bdt_events["H1PNetMass"] = events["bbFatJetPNetMassLegacy"][0]
        bdt_events["H2PNetMass"] = events["bbFatJetPNetMassLegacy"][1]
        bdt_events["H1Msd"] = events["bbFatJetMsd"][0]
        bdt_events["H1TXbb"] = events[f"bbFatJetPNetTXbb{legacy_label}"][0]
        bdt_events["H2TXbb"] = events[f"bbFatJetPNetTXbb{legacy_label}"][1]
        bdt_events["weight"] = events["finalWeight"].to_numpy()

        bdt_events["hlt"] = np.any(
            np.array(
                [events[trigger][0] for trigger in postprocessing.HLTs[year] if trigger in events]
            ),
            axis=0,
        )
        mask_hlt = bdt_events["hlt"] == 1

        # masks
        mask_presel = (
            (bdt_events["H1Msd"] > 40)
            & (bdt_events["H1Pt"] > 300)
            & (bdt_events["H2Pt"] > 250)
            & (bdt_events["H1TXbb"] > 0.8)
        )
        mask_mass = (bdt_events["H2PNetMass"] > 50) & (bdt_events["H2PNetMass"] < 250)
        bdt_events = bdt_events[(mask_mass) & (mask_hlt) & (mask_presel)]

        columns = [
            "bdt_score",
            "H1TXbb",
            "H2TXbb",
            "H1PNetMass",
            "H2PNetMass",
            "weight",
            "H1Pt",
            "H2Pt",
        ]
        bdt_events_dict[key] = bdt_events[columns]
    return bdt_events_dict

In [ ]:
data_dir = "24May24_v12_private_signal"
input_dir = f"/ceph/cms/store/user/cmantill/bbbb/skimmer/{data_dir}"

samples_run3 = {
    "2022EE": {
        "hh4b": ["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV?"],
    },
    "2022": {
        "hh4b": ["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV?"],
    },
    "2023": {
        "hh4b": ["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV?"],
    },
    "2023BPix": {
        "hh4b": ["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV?"],
    },
}

bdt_events_dict_year = {}
for year in samples_run3:
    events = HH4b.postprocessing.load_run3_samples(
        input_dir=input_dir,
        year=year,
        legacy=True,
        samples_run3=samples_run3,
        reorder_txbb=True,
        txbb=f"bbFatJetPNetTXbb{legacy_label}",
    )
    bdt_events_dict_year[year] = get_dataframe(events, year, bdt_model_name, bdt_config)

In [ ]:
events_combined, scaled_by = postprocessing.combine_run3_samples(
    bdt_events_dict_year,
    ["hh4b"],
    bg_keys=["ttbar"],
    scale_processes={},
    years_run3=bdt_events_dict_year.keys(),
)

In [ ]:
labels = {
    "hh4b": "HH (4b)",
}
bdt_axis = hist.axis.Variable(list(np.arange(0.99, 1, 0.0001)), name="BDT score")
txbb1_axis = hist.axis.Variable(list(np.arange(0.99, 1, 0.0001)), name=r"Jet 1 $T_{Xbb}$")
txbb2_axis = hist.axis.Variable(list(np.arange(0, 1, 0.0001)), name=r"Jet 2 $T_{Xbb}$")


for key, events in events_combined.items():

    h_xbb1_bdt = hist.Hist(txbb1_axis, bdt_axis, storage=hist.storage.Weight())
    h_xbb1_bdt_bin1 = hist.Hist(txbb1_axis, bdt_axis, storage=hist.storage.Weight())
    h_xbb1_bdt_bin2 = hist.Hist(txbb1_axis, bdt_axis, storage=hist.storage.Weight())
    h_xbb1_bdt_bin3 = hist.Hist(txbb1_axis, bdt_axis, storage=hist.storage.Weight())

    h_xbb1 = hist.Hist(txbb1_axis, storage=hist.storage.Weight())
    h_xbb1_bin1 = hist.Hist(txbb1_axis, storage=hist.storage.Weight())
    h_xbb1_bin2 = hist.Hist(txbb1_axis, storage=hist.storage.Weight())
    h_xbb1_bin3 = hist.Hist(txbb1_axis, storage=hist.storage.Weight())

    mask_t2xbb = events["H2TXbb"] > 0.975
    mask_bin1 = (events["H2TXbb"] > 0.975) & (events["bdt_score"] > 0.98)
    mask_bin2 = (~mask_bin1) & (
        ((events["H2TXbb"] > 0.975) & (events["bdt_score"] > 0.88))
        | ((events["H2TXbb"] > 0.92) & (events["bdt_score"] > 0.98))
    )
    mask_bin3 = (
        (~mask_bin1) & (~mask_bin2) & ((events["H2TXbb"] > 0.92) & (events["bdt_score"] > 0.88))
    )

    h_xbb1_bdt.fill(
        events["H1TXbb"],
        events["bdt_score"],
    )
    h_xbb1_bdt_bin1.fill(
        events["H1TXbb"][mask_bin1],
        events["bdt_score"][mask_bin1],
    )
    h_xbb1_bdt_bin2.fill(
        events["H1TXbb"][mask_bin2],
        events["bdt_score"][mask_bin2],
    )
    h_xbb1_bdt_bin3.fill(
        events["H1TXbb"][mask_bin3],
        events["bdt_score"][mask_bin3],
    )

    h_xbb1.fill(events["H1TXbb"])
    h_xbb1_bin1.fill(events["H1TXbb"][mask_bin1])
    h_xbb1_bin2.fill(events["H1TXbb"][mask_bin2])
    h_xbb1_bin3.fill(events["H1TXbb"][mask_bin3])

    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    hep.hist2dplot(h_xbb1_bdt, ax=ax)
    ax.set_title(key)

    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    hep.hist2dplot(h_xbb1_bdt_bin1, ax=ax)
    ax.set_title(f"{key}, ggF category 1")

    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    hep.hist2dplot(h_xbb1_bdt_bin2, ax=ax)
    ax.set_title(f"{key}, ggF category 2")

    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    hep.hist2dplot(h_xbb1_bdt_bin3, ax=ax)
    ax.set_title(f"{key}, ggF category 3")

    fig = plt.figure(figsize=(10, 8))
    main_ax_artists, sublot_ax_arists = h_xbb1_bin1.plot_ratio(
        h_xbb1,
        rp_ylabel=r"Efficiency",
        rp_num_label="ggF Category 1",
        rp_denom_label="Preselection",
        rp_uncert_draw_type="line",  # line or bar
        rp_uncertainty_type="efficiency",
    )

    fig = plt.figure(figsize=(10, 8))
    main_ax_artists, sublot_ax_arists = h_xbb1_bin2.plot_ratio(
        h_xbb1,
        rp_ylabel=r"Efficiency",
        rp_num_label="ggF Category 2",
        rp_denom_label="Preselection",
        rp_uncert_draw_type="line",  # line or bar
        rp_uncertainty_type="efficiency",
    )

    fig = plt.figure(figsize=(10, 8))
    main_ax_artists, sublot_ax_arists = h_xbb1_bin3.plot_ratio(
        h_xbb1,
        rp_ylabel=r"Efficiency",
        rp_num_label="ggF Category 3",
        rp_denom_label="Preselection",
        rp_uncert_draw_type="line",  # line or bar
        rp_uncertainty_type="efficiency",
    )

In [ ]:
np.quantile(events["H1TXbb"][mask_bin1], q=[0.16, 0.33, 0.5, 0.84])

In [ ]:
year = "2023BPix"
with open(f"{package_path}/corrections/data/txbb_sfs/{year}/sf_txbbv11_Jun14.json") as f:
    txbb_sf_old = json.load(f)
with open(f"{package_path}/corrections/data/txbb_sfs/{year}/sf_txbbv11_Jun26_freezeSFs.json") as f:
    txbb_sf = json.load(f)
with open(
    f"{package_path}/corrections/data/txbb_sfs/{year}/sf_txbbv11_Jun27_freezeSFs_finerWPs.json"
) as f:
    txbb_sf_fine = json.load(f)
with open(
    f"{package_path}/corrections/data/txbb_sfs/{year}/sf_txbbv11_Jun29_freezeSFs_zoomedInWPs.json"
) as f:
    txbb_sf_zoom = json.load(f)
with open(
    f"{package_path}/corrections/data/txbb_sfs/{year}/sf_txbbv11_Jul3_freezeSFs_newPt.json"
) as f:
    txbb_sf_new_pt = json.load(f)

In [ ]:
txbb_sf_new = {}

ptbins = np.array([200, 250, 300, 400, 500, 100000])

for i in range(len(ptbins) - 1):
    txbb_sf_new[f"WP3_pt{ptbins[i]}to{ptbins[i+1]}"] = txbb_sf_zoom[
        f"WP2_pt{ptbins[i]}to{ptbins[i+1]}"
    ]
    txbb_sf_new[f"WP4_pt{ptbins[i]}to{ptbins[i+1]}"] = txbb_sf_fine[
        f"WP2_pt{ptbins[i]}to{ptbins[i+1]}"
    ]
    txbb_sf_new[f"WP5_pt{ptbins[i]}to{ptbins[i+1]}"] = txbb_sf_fine[
        f"WP3_pt{ptbins[i]}to{ptbins[i+1]}"
    ]
    txbb_sf_new[f"WP6_pt{ptbins[i]}to{ptbins[i+1]}"] = txbb_sf_fine[
        f"WP4_pt{ptbins[i]}to{ptbins[i+1]}"
    ]

ptbins_new = np.array([200, 400, 100000])

for i in range(len(ptbins_new) - 1):
    txbb_sf_new[f"WP1_pt{ptbins_new[i]}to{ptbins_new[i+1]}"] = txbb_sf_new_pt[
        f"WP1_pt{ptbins_new[i]}to{ptbins_new[i+1]}"
    ]
    txbb_sf_new[f"WP2_pt{ptbins_new[i]}to{ptbins_new[i+1]}"] = txbb_sf_new_pt[
        f"WP2_pt{ptbins_new[i]}to{ptbins_new[i+1]}"
    ]

# with open(
#     f"{package_path}/corrections/data/txbb_sfs/{year}/sf_txbbv11_Jul3_freezeSFs_combinedWPs.json",
#     "w",
# ) as f:
#     json.dump(txbb_sf_new, f, indent=4)

In [ ]:
y_new, yerr_low_new, yerr_high_new = {}, {}, {}
wps = {
    "WP1": [0.998, 1],
    "WP2": [0.995, 0.998],
    "WP3": [0.99, 0.995],
    "WP4": [0.975, 0.99],
    "WP5": [0.95, 0.975],
    "WP6": [0.92, 0.95],
}
for wp in range(0, 2):
    y_new[wp] = []
    yerr_low_new[wp] = []
    yerr_high_new[wp] = []
    for i in range(len(ptbins_new) - 1):
        y_new[wp].append(
            txbb_sf_new[f"WP{wp+1}_pt{ptbins_new[i]}to{ptbins_new[i+1]}"]["final"]["central"]
        )
        yerr_low_new[wp].append(
            txbb_sf_new[f"WP{wp+1}_pt{ptbins_new[i]}to{ptbins_new[i+1]}"]["final"]["low"]
        )
        yerr_high_new[wp].append(
            txbb_sf_new[f"WP{wp+1}_pt{ptbins_new[i]}to{ptbins_new[i+1]}"]["final"]["high"]
        )
for wp in range(2, 6):
    y_new[wp] = []
    yerr_low_new[wp] = []
    yerr_high_new[wp] = []
    for i in range(len(ptbins) - 1):
        y_new[wp].append(txbb_sf_new[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["central"])
        yerr_low_new[wp].append(
            txbb_sf_new[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["low"]
        )
        yerr_high_new[wp].append(
            txbb_sf_new[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["high"]
        )

plt.figure()
# horizaontal line at 1
plt.axhline(1, color="gray", linestyle="--", alpha=0.5)
# vertical line at each ptbin
for ptbin in ptbins[:-1]:
    plt.axvline(ptbin, color="gray", linestyle="-", alpha=0.5)
for wp in range(0, 2):
    plt.errorbar(
        y=y_new[wp],
        x=ptbins_new[:-1] + (wp + 1) * 5,
        yerr=[yerr_low_new[wp], yerr_high_new[wp]],
        fmt="o",
        label=f"WP{wp+1} {wps[f'WP{wp+1}']}",
    )
for wp in range(2, 6):
    plt.errorbar(
        y=y_new[wp],
        x=ptbins[:-1] + (wp + 1) * 5,
        yerr=[yerr_low_new[wp], yerr_high_new[wp]],
        fmt="o",
        label=f"WP{wp+1} {wps[f'WP{wp+1}']}",
    )
plt.xlabel("$p_T (j)$ [GeV]")
plt.ylabel("SF (flvB)")
plt.ylim([0, 2])
plt.legend(title=year)
plt.savefig(f"new_txbb_sf_{year}.pdf")

In [ ]:
y_new, yerr_low_new, yerr_high_new = [], [], []
wps = {
    "WP1": [0.998, 1],
    "WP2": [0.995, 0.998],
    "WP3": [0.99, 0.995],
    "WP4": [0.975, 0.99],
    "WP5": [0.95, 0.975],
    "WP6": [0.92, 0.95],
}
for i in range(len(ptbins) - 1):
    y_new.append([])
    yerr_low_new.append([])
    yerr_high_new.append([])
    for wp in reversed(range(2, 6)):
        y_new[i].append(txbb_sf_new[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["central"])
        yerr_low_new[i].append(
            txbb_sf_new[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["low"]
        )
        yerr_high_new[i].append(
            txbb_sf_new[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["high"]
        )
    for wp in reversed(range(0, 2)):
        for j in range(len(ptbins_new) - 1):
            if ptbins[i] >= ptbins_new[j] and ptbins[i+1] <= ptbins_new[j+1]:
                y_new[i].append(
                    txbb_sf_new[f"WP{wp+1}_pt{ptbins_new[j]}to{ptbins_new[j+1]}"]["final"]["central"]
                )
                yerr_low_new[i].append(
                    txbb_sf_new[f"WP{wp+1}_pt{ptbins_new[j]}to{ptbins_new[j+1]}"]["final"]["low"]
                )
                yerr_high_new[i].append(
                    txbb_sf_new[f"WP{wp+1}_pt{ptbins_new[j]}to{ptbins_new[j+1]}"]["final"]["high"]
                )
                break

# make 5 subfigures
fig, axs = plt.subplots(5, 1, figsize=(8, 30))
for i in range(len(ptbins) - 1):
    plt.sca(axs[i])
    plt.axhline(1, color="gray", linestyle="--", alpha=0.5)
    x = [np.mean(wps[wp]) for wp in ["WP6", "WP5", "WP4", "WP3", "WP2", "WP1"]]
    xerr = [(wps[wp][1] - wps[wp][0]) / 2 for wp in ["WP6", "WP5", "WP4", "WP3", "WP2", "WP1"]]
    plt.errorbar(
        y=y_new[i][:-1],
        x=x[:-1],
        xerr=xerr[:-1],
        yerr=[yerr_low_new[i][:-1], yerr_high_new[i][:-1]],
        fmt="o",
        label=f"$[{ptbins[i]}, {ptbins[i+1]}]$ GeV".replace("100000", "\inf"),
    )
    j = 0 if i < 3 else 1
    plt.errorbar(
        y=y_new[i][-2:],
        x=x[-2:],
        xerr=xerr[-2:],
        yerr=[yerr_low_new[i][-2:], yerr_high_new[i][-2:]],
        fmt="o",
        label=f"$[{ptbins_new[j]}, {ptbins_new[j+1]}]$ GeV".replace("100000", "\inf"),
    )
    extended_wps = np.array([0.92, 0.95, 0.975, 0.99, 0.995, 0.998, 1])
    extended_y = np.array(y_new[i] + [y_new[i][-1]])
    extended_yerr_low = np.array(yerr_low_new[i] + [yerr_low_new[i][-1] * 3])
    extended_yerr_high = np.array(yerr_high_new[i] + [yerr_high_new[i][-1] * 3])
    plt.fill_between(
        extended_wps[:-2],
        extended_y[:-2] - extended_yerr_low[:-2],
        extended_y[:-2] + extended_yerr_high[:-2],
        alpha=0.2,
        step="post",
    )
    plt.fill_between(
        extended_wps[-3:-1],
        extended_y[-3] - extended_yerr_low[-3],
        extended_y[-3] + extended_yerr_high[-3],
        alpha=0.2,
    )
    # reset color to orange
    # print(plt.rcParams['axes.prop_cycle'].by_key()['color'])
    # plt.gca().set_prop_cycle(None)
    plt.plot()
    plt.fill_between(
        extended_wps[-2:],
        extended_y[-2:] - extended_yerr_low[-2:],
        extended_y[-2:] + extended_yerr_high[-2:],
        alpha=0.2,
        color="#f89c20",
    )
    plt.xlabel("$T_{Xbb}$")
    plt.ylabel("SF (flvB)")
    plt.ylim([0, 2])
    plt.xlim([0.92, 1])
    plt.legend(title=year)
plt.savefig(f"new_txbb_sf_3x_{year}.pdf")

In [ ]:
y_old, yerr_low_old, yerr_high_old = [], [], []
wps = {
    "WP1": [0.975, 1],
    "WP2": [0.95, 0.975],
    "WP3": [0.92, 0.95],
}
for wp in range(0, 3):
    y_old.append([])
    yerr_low_old.append([])
    yerr_high_old.append([])
    for i in range(len(ptbins) - 1):
        y_old[wp].append(txbb_sf[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["central"])
        yerr_low_old[wp].append(txbb_sf[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["low"])
        yerr_high_old[wp].append(txbb_sf[f"WP{wp+1}_pt{ptbins[i]}to{ptbins[i+1]}"]["final"]["high"])

plt.figure()
# horizaontal line at 1
plt.axhline(1, color="gray", linestyle="--", alpha=0.5)
# vertical line at each ptbin
for ptbin in ptbins[:-1]:
    plt.axvline(ptbin, color="gray", linestyle="-", alpha=0.5)
for wp in range(0, 3):
    plt.errorbar(
        y=y_old[wp],
        x=ptbins[:-1] + (wp + 3) * 5,
        yerr=[yerr_low_old[wp], yerr_high_old[wp]],
        fmt="o",
        label=f"WP{wp+1} {wps[f'WP{wp+1}']}",
    )
plt.xlabel("$p_T (j)$ [GeV]")
plt.ylabel("SF (flvB)")
plt.ylim([0, 2])
plt.legend(title=year)
plt.savefig(f"old_txbb_sf_{year}.pdf")